Infiltrationsbestimmung nach Green und Ampt

Mit dem Green-Ampt Modell lässt sich die Infiltrationsrate in eine Bodensäule mit einem Querschnitt von 1 m² als exponentielle Funktion der Zeit darstellen.

$ Formel §

In [87]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact,interactive, fixed, interact_manual, Button, Layout
from IPython.display import clear_output


Messdaten für die Infiltration [l/s] in einen 1 m² großen Bodenquerschnitt können als csv-Datei durch Eingabe des vollständigen Dateipfades in das nachstehende Eingabefenster eingelesen werden.

Im zweiten Eingabefenster ist der Spaltenname der Zeitdatenreihe einzufügen.

In [88]:


class GreenAmptModel:                                                                   # Klasse mit Methoden zur Berechnung und Visualisierung der Infiltration nach Green-Ampt (1911)

    def __init__(self):             
        self.reference_lines = []                                                                       # Initialisierung von Parametern bzw. Listen, in denen später berechnete
        self.ref_line_params = []                                                                       # Werte gespeichert werden können.
        self.short_time_ref = []                                                                        # Durch die Initialisierung sind die mit "self" markierten Variablen für alle
        self.short_time_params = []                                                                     # Methoden in der Klasse: GreenAmptModel sichtbar.
        self.infiltration_rate_ref = []
        self.infiltration_ref = []


    def t_for_It(self, I_t, d_theta, d_h, Ks=0.3):                                                      # Methode zur Berechnung der Zeitschritte zwischen einzelnen Infiltrationsschrittenb(t)
            return (I_t - d_theta * d_h * np.log(1+ (I_t/(d_theta*d_h))))/Ks                                    # Ausgabe: np.array

    def long_time_approximation(self, K, x):                                                            # Methode zur Berechnung der Langzeitannäherung (i_t_long) an die Infiltrationsrate (i_t)
        it_long = np.array([K]*len(x))                                                                          # Ausgabe: np.array
        return it_long

    def d_h(self, h0=0, hf=-60):                                                                        # Methoden zur Berechnung einer Differenz zwischen zwei Werten
        return h0-hf                                                                                            # Ausgabe: Float

    def d_theta(self, theta_0=0.44, theta_i=0.1):
        return theta_0 - theta_i
    
    def infiltration_rate(self, new_t, t, I_t):                                                         # Methode zur Berechnung der Infiltationsate 
        y2 = np.interp(new_t, t, I_t)                                                                           # Ausgabe: np.array
        i_t = np.diff(y2)
        #self.i_t = i_t
        return i_t
    
    def read_datastring(self, y_data_string='', x_data_string=''):                                      # Methode zum Einlesen von Messdaten
        if y_data_string or x_data_string != '':                                                                # Die Messdaten werden als String eingelesen und in Listen umgewandelt
            y_data = y_data_string.split(',')                                                                   # Ausgabe: 2 Listen mit Funktionswerten
            x_data = y_data_string.split(',')
            return y_data, x_data


    def plot_scenario(                                                                                  # Methode zum Plotten der Daten
            self, hf_h0=(-60,0), thetai_theta0=(0.2,0.44), Ks=4, i_data_string='',
            ti_data_string='', I_data_string='', tI_data_string='' 
            ):

        h0 = hf_h0[1]                                                                                   # Vom Benutzer eingelesene Werte werden in Variablen gespeichert
        hf = hf_h0[0]
        theta_i = thetai_theta0[0]
        theta_0 = thetai_theta0[1]
        
                                                                                                        # Aus den definierten Methoden werden Daten generiert
        delta_theta = self.d_theta(theta_0, theta_i)                                                        # delta theta (type: Float)
        delta_h = self.d_h(h0, hf)                                                                          # delta h (type: Int)
        I_t = np.arange(0.1, 260, 0.1)                                                                      # Infiltrationsdaten (type: np.array)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks)                                                    # Zeitschritte für die Infiltration (type: np.array)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)                                       # Zeitschritte für die Infiltration auf 1. Dezimalstelle gerundet (type: np.array)
        self.i_t = self.infiltration_rate(self.new_t, t, I_t)                                               # Infiltrationsrate (type: np.array)
        i_t_long = self.long_time_approximation(Ks, self.new_t)                                             # Langzeitannäherung der Infiltrationsrate ~ Ks (type: np.array)


        fig, (ax1, ax2) = plt.subplots(2, figsize=(12,12))                                              # Eine Abbildung für die Plots wird erstellt


        for ref, params in zip(self.infiltration_ref, self.ref_line_params):                            # In on_button_click() definierte Referenzlinien für I_t werden geplottet
            ref_x, ref_y = ref
            ax1.plot(ref_x, ref_y, linestyle='-.', alpha=0.6, 
                     label='I(t): hf:{}, h0:{}, theta_i:{}, theta_0:{}, Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)
        
        for ref, params in zip(self.infiltration_rate_ref, self.ref_line_params):                       # In on_button_click() definierte Referenzlinien für I_t werden geplottet
            ref_x, ref_y = ref
            ax2.plot(ref_x, ref_y, linestyle='-.', alpha=0.6,
                      label='i(t): hf:{} h0:{} theta_i:{} theta_0:{} Ks:{}'.format(params[0], params[1], params[2], params[3], params[4]), linewidth=2.5)


        ax1.plot(t, I_t, color='black', label='I(t)', linewidth=1)                                      # Plotten der Infiltrationsdaten auf einer Achse 
        
        ax1.set_title(label='Green-Ampt Infiltrationsmodell', fontsize=20, loc= 'center')                       # Grenzen für die x- und y-Achse festlegen
        ax1.set_ylim(0.1, max(I_t))
        ax1.set_xlim(0, 185)
        ax1.set_xlabel('Zeit [min]', fontsize=10)                                                               # Achsenbeschreibung hinzufügen
        ax1.set_ylabel('Infiltration [mm/min]', fontsize=10)
        ax1.grid(which = "major", linewidth = 1)                                                                # Gitternetz hinzufügen
        ax1.grid(which = "minor", linewidth = 0.2)
        ax1.minorticks_on()
        ax1.legend(loc='best')                                                                                  # Legende einblenden


        ax2.plot(self.new_t[1:-5], self.i_t[0:-5]*100, color='black', label='i(t)', linewidth=1)        # Plotten der Infiltrationsrate auf einer weiteren Achse
        ax2.plot(self.new_t, i_t_long, 'red', linestyle='--',                                           # Plotten der Langzeitnäherung auf derselben Achse
                  label='Long time approximation ~ Ks [mm/min]')
  
        ax2.set_ylim(0.1, 150)                                                                                  # Grenzen für die x- und y-Achse festlegen
        ax2.set_xlim(0, max(self.new_t))
        ax2.set_yscale('log')                                                                                   # y-Achsenskalierung logarithmisch setzen
        ax2.set_xlabel('Zeit [min]', fontsize=10)                                                               # Achsenbeschreibung hinzufügen
        ax2.set_ylabel('Infiltrationsrate [mm/min]', fontsize=10)
        ax2.grid(which = "major", linewidth = 1)                                                                # Gitternetz hinzufügen
        ax2.grid(which = "minor", linewidth = 0.2)
        ax2.minorticks_on()
        ax2.legend(loc='best')                                                                                  # Legende einblenden

                                                                                                       # Plotten von zusätzlichen, vom Benutzer eingegebenen Messdaten
                                                                                                          ## für die Infiltrationsrate
        if i_data_string or ti_data_string != '':                                                               # Wurden Messdaten übergeben? Wenn, ja:
            i_data, ti_data = self.read_datastring(i_data_string, ti_data_string)                               # Umwandeln von str -> list
            ax2.plot(ti_data, i_data, color='blue', linestyle='', marker='+')                                   # Plotten

                                                                                                        ## für die Infiltration
        if I_data_string or tI_data_string != '':                                                               # Wurden Messdaten übergeben? Wenn, ja:
            I_data, tI_data = self.read_datastring(I_data_string, tI_data_string)                               # Umwandeln von str -> list
            ax1.plot(tI_data, I_data, color='blue', linestyle='', marker='+')                                   # Plotten

        if self.on_sfbutton_click(self.sf_button) == True:                                              # Wenn der Button gedrückt ist, wird die Abbildung gespeichert
            fig.savefig('Green_Ampt_Infiltration.png')

        plt.show()

    def on_button_click(self, button):
  
        h0_val = self.hf_h0.value[1]                                                                    # Speichern der über "Widgets" eingelesenen Werte in Variablen
        hf_val = self.hf_h0.value[0]
        theta_i_val = self.thetai_theta0.value[0]
        theta_0_val = self.thetai_theta0.value[1]
        Ks_val = self.Ks_slider.value

        delta_theta = (self.d_theta(theta_0_val, theta_i_val))                                          # Berechnen der Werte mittels oben definierter Methoden
        delta_h = self.d_h(h0_val, hf_val)
        I_t = np.arange(0, 260, 0.1)
        t = self.t_for_It(I_t, delta_theta, delta_h, Ks_val)
        self.new_t = np.arange(round(t[0], 1), round(t[-1], 1), 0.01)
        i_t = self.infiltration_rate(self.new_t, t, I_t)

        # Store the current short time approximation data as a reference line
        self.reference_lines.append((t, I_t))                                                           # Speichern der gerade vom Benutzer eingestellten Funktionsverläufe als np.arrays
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))                 # ebenfalls für die Parameter
        self.reference_lines.append((self.new_t, i_t))
        self.ref_line_params.append((h0_val, hf_val, theta_0_val, theta_i_val, Ks_val))
        self.infiltration_ref.append((t, I_t))
        self.infiltration_rate_ref.append((self.new_t[1:-5], self.i_t[:-5]*100))

        clear_output(wait=True)                                                                         # Alten Plot löschen und neuen inklusive Referenzlinien Plotten
        display(self.interactive_plot)
        display(button)
        display(self.sf_button)

    def on_sfbutton_click(self, sf_button):
        return True                                                                                     # Übergibt einen Wahrheitswert, wenn der Button gedrückt ist


    def create_interactive_plot(self):
        
        button = Button(description="Referenzlinie Speichern",                                                      # Erstellen eines Buttons zum Speichern der Referenzlinien
                         layout=Layout(width='20%', height='40px'), button_style='info')
        button.on_click(self.on_button_click)                                                                               # Funktion für den Fall des 'gedrückten buttons'

        self.sf_button = Button(description="Abbildung Speichern",                                                  #Button zum Speichern der Abbildung
                                layout=Layout(width='20%', height='40px'), button_style='info')                                                  
        self.sf_button.on_click(self.on_sfbutton_click)

        self.hf_h0 = widgets.IntRangeSlider(                                                            # Ein IntRangeSlider zum Einlesen von hf und h0 über den Benutzer(übergibt 2 Werte als tuple)
            value=[-600, 0], min=-1000, max=100, step=1,
            description=('hf - h0 [mm]'),style = {'description_width': 'initial'}
            )
        self.thetai_theta0 = widgets.FloatRangeSlider(                                                  # Ein FloatRangeSlider für theta i und theta 0
            value=[0.1, 0.4], min=0, max=0.5, step=0.01,
            description=('theta i - theta 0 [-]'), style = {'description_width': 'initial'}
            )
        self.Ks_slider = widgets.FloatSlider(                                                           # Ein FloatSlider für hf und h0 (übergibt einen Float)
            value=0.5, min=0.01, max=5, step=0.01,
            description='Ks [mm/min]' , style = {'description_width': 'initial'}
            )

    
        self.I_data_string = widgets.Textarea(                                                          # 4 Textfelder zum Einlesen von Messdaten (übergibt jeweils einen String)
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
            description='Infiltrationsmessdaten', style = {'description_width': 'initial'}
            )
        
        self.tI_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Zeitdaten für I(t)', style = {'description_width': 'initial'}
              )

        self.i_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Infiltrationsrate', style = {'description_width': 'initial'}
              )
        
        self.ti_data_string = widgets.Textarea(
            placeholder='Daten Bitte mit . als Dezimaltrennzeichen und , als Trennzeichen eingeben',
              description='Zeitdaten für i(t)', style = {'description_width': 'initial'}
              )

        self.interactive_plot = interactive(                                                             # Verknüpfen der interaktiven Eingabewidgets mit dem Plot
            self.plot_scenario, hf_h0 = self.hf_h0, thetai_theta0 = self.thetai_theta0, Ks=self.Ks_slider, 
            i_data_string=self.i_data_string, ti_data_string=self.ti_data_string, 
            I_data_string=self.I_data_string, tI_data_string=self.tI_data_string
            )
        
        display(self.interactive_plot)                                                                   # Aufruf von Plot und Button        
        display(button)
        display(self.sf_button)



In [89]:
GreenAmptModel().create_interactive_plot()


interactive(children=(IntRangeSlider(value=(-600, 0), description='hf - h0 [mm]', min=-1000, style=SliderStyle…

Button(button_style='info', description='Referenzlinie Speichern', layout=Layout(height='40px', width='20%'), …

Button(button_style='info', description='Abbildung Speichern', layout=Layout(height='40px', width='20%'), styl…